# Data Science Nigeria Staff Promotion Algorithm - XGBoostn Complex

In [0]:
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import pandas as pd
from sklearn import preprocessing

from xgboost import plot_importance
import numpy as np
from sklearn.metrics import confusion_matrix


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
df=pd.read_csv('gdrive/My Drive/DSN Project/train.csv')
df.set_index('EmployeeNo',inplace=True)

In [0]:
dfvalidation = pd.read_csv('gdrive/My Drive/DSN Project/test.csv')
dfvalidation.set_index('EmployeeNo',inplace=True)
dfvalidation['Promoted_or_Not'] = 5
dfvalidation.head()


,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
EmployeeNo,,,,,,,,,,,,,,,,,,
YAK/S/00005,Information Technology and Solution Support,First Degree or HND,Male,Agency and others,2,1976,7.5,2017,0,0,65,FCT,Yes,Married,No,No,1,5
YAK/S/00011,Information Technology and Solution Support,NaN,Male,Direct Internal process,2,1991,0.0,2018,0,0,69,OGUN,Yes,Married,No,No,1,5
YAK/S/00015,Research and Innovation,"MSc, MBA and PhD",Male,Direct Internal process,2,1984,7.5,2012,0,0,76,KANO,Yes,Married,No,No,1,5
YAK/S/00016,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1984,2.5,2009,0,0,52,RIVERS,No,Single,No,No,1,5
YAK/S/00017,Information Technology and Solution Support,First Degree or HND,Male,Agency and others,2,1983,7.5,2014,1,0,69,FCT,Yes,Married,No,No,0,5


In [0]:

# Treating the missing values of education as a separate category
dfvalidation['Qualification'] = dfvalidation['Qualification'].replace(np.NaN, 'NA')

#Convert Year of birth to Age
dfvalidation = dfvalidation.rename(columns = {'Year_of_birth': 'Age'})
dfvalidation['Age'] = 2019 - dfvalidation['Age']

#Convert the 'Year_of_recruitment' to number of years in company by subtracting from the year 2019
dfvalidation = dfvalidation.rename(columns ={'Year_of_recruitment': 'Years_in_company'})
dfvalidation['Years_in_company'] = 2019 - dfvalidation['Years_in_company']

#We replace all entries of 'More than 5' with 6 and convert the column type to integer
dfvalidation['No_of_previous_employers'] = dfvalidation['No_of_previous_employers'].replace(regex = 'More than 5', value = 6)
dfvalidation['No_of_previous_employers'] = dfvalidation['No_of_previous_employers'].astype(int)


##One hot encoding
dfvalidation = pd.concat([dfvalidation[['Trainings_Attended','Age','Last_performance_score','Years_in_company','Targets_met','Previous_Award','Training_score_average','No_of_previous_employers']],
               pd.get_dummies(dfvalidation['Division'],drop_first = True),pd.get_dummies(dfvalidation['Qualification'],drop_first = True),pd.get_dummies(dfvalidation['Gender'],drop_first = True),pd.get_dummies(dfvalidation['Channel_of_Recruitment'],drop_first = True),pd.get_dummies(dfvalidation['Foreign_schooled'],drop_first = True),pd.get_dummies(dfvalidation['Marital_Status'],drop_first = True),pd.get_dummies(dfvalidation['Past_Disciplinary_Action'],drop_first = True),pd.get_dummies(dfvalidation['State_Of_Origin'],drop_first = True),pd.get_dummies(dfvalidation['Previous_IntraDepartmental_Movement'],drop_first = True)],axis=1)

In [0]:
##Merging the train and test dataset
dfmerged = pd.concat([df,dfvalidation])
dfmerged.shape

(54808, 18)

In [0]:
dfmerged.columns

Index(['Division', 'Qualification', 'Gender', 'Channel_of_Recruitment',
       'Trainings_Attended', 'Year_of_birth', 'Last_performance_score',
       'Year_of_recruitment', 'Targets_met', 'Previous_Award',
       'Training_score_average', 'State_Of_Origin', 'Foreign_schooled',
       'Marital_Status', 'Past_Disciplinary_Action',
       'Previous_IntraDepartmental_Movement', 'No_of_previous_employers',
       'Promoted_or_Not'],
      dtype='object')

In [0]:
# Treating the missing values of education as a separate category
dfmerged['Qualification'] = dfmerged['Qualification'].replace(np.NaN, 'NA')

#Convert Year of birth to Age
dfmerged = dfmerged.rename(columns = {'Year_of_birth': 'Age'})
dfmerged['Age'] = 2019 - dfmerged['Age']

#Convert the 'Year_of_recruitment' to number of years in company by subtracting from the year 2019
dfmerged = dfmerged.rename(columns ={'Year_of_recruitment': 'Years_in_company'})
dfmerged['Years_in_company'] = 2019 - dfmerged['Years_in_company']

#We replace all entries of 'More than 5' with 6 and convert the column type to integer
dfmerged['No_of_previous_employers'] = dfmerged['No_of_previous_employers'].replace(regex = 'More than 5', value = 6)
dfmerged['No_of_previous_employers'] = dfmerged['No_of_previous_employers'].astype(int)



In [0]:
### Dividing age into different bands as this company has 9 verticals and they are finding promotion for manager and below.So didvided
###age into 4 different bands
'''
def findBand(age):
    if age <=29:
        return 1
    elif age>29 and age<=33:
        return 2
    elif age>33 and age<=39:
        return 3
    else:
        return 4

dfmerged['band'] = dfmerged['age'].apply(findBand)
'''

In [0]:
len(dfmerged.columns)

18

In [0]:
##One hot encoding
dfmerged = pd.concat([dfmerged[['Trainings_Attended','Age','Last_performance_score','Years_in_company','Targets_met','Previous_Award','Training_score_average','No_of_previous_employers', 'Promoted_or_Not']],
               pd.get_dummies(dfmerged['Division'],drop_first = True),pd.get_dummies(dfmerged['Qualification'],drop_first = True),pd.get_dummies(dfmerged['State_Of_Origin'],drop_first = True),pd.get_dummies(dfmerged['Gender'],drop_first = True),pd.get_dummies(dfmerged['Channel_of_Recruitment'],drop_first = True),pd.get_dummies(dfmerged['Foreign_schooled'],drop_first = True),pd.get_dummies(dfmerged['Marital_Status'],drop_first = True),pd.get_dummies(dfmerged['Past_Disciplinary_Action'],drop_first = True),pd.get_dummies(dfmerged['Previous_IntraDepartmental_Movement'],drop_first = True)],axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [0]:
pd.DataFrame(dfmerged.columns)

,0
0,Trainings_Attended
1,Age
2,Last_performance_score
3,Years_in_company
4,Targets_met
5,Previous_Award
6,Training_score_average
7,No_of_previous_employers
8,Promoted_or_Not
9,Commercial Sales and Marketing


In [0]:
###Generating all possible pair of interactions between 2 pair of columns.
##Then removing those columns 

from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

Y = dfmerged.Promoted_or_Not
X = dfmerged.drop(['Promoted_or_Not'],1)
 


def add_interactions(df):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns)+['_'.join(x) for x in combos]
    
    scaler = MinMaxScaler()
    scaler.fit(df)
    df = scaler.transform(df)
    
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    df = poly.fit_transform(df)
    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indices = [i for i,x in enumerate(list((df==0).all())) if x]
    df= df.drop(df.columns[noint_indices], axis=1)
    
    return df
X = add_interactions(X)
dfvalidation = add_interactions(dfvalidation)
X.shape

(54808, 342)

In [0]:
###Seperate out the training and testing dataset

X.index = Y.index
X['Promoted_or_Not'] = Y

X_validation = X[38312:]
X = X[0:38312]

In [0]:
Y = X.Promoted_or_Not
X = X.drop(['Promoted_or_Not'],1)
 

seed = 42
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [0]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(30649, 342) (30649,)
(7663, 342) (7663,)


In [0]:
duplicate_columns = X_train.columns[X_train.columns.duplicated()]
print(duplicate_columns)
X_train = X_train.drop(duplicate_columns, axis = 1)
X_test = X_test.drop(duplicate_columns, axis = 1)
X = X.drop(duplicate_columns, axis = 1)


Index(['Yes', 'Yes', 'Trainings_Attended_Yes', 'Trainings_Attended_Yes',
       'Age_Yes', 'Age_Yes', 'Last_performance_score_Yes',
       'Last_performance_score_Yes', 'Years_in_company_Yes',
       'Years_in_company_Yes', 'Targets_met_Yes', 'Targets_met_Yes',
       'Previous_Award_Yes', 'Previous_Award_Yes',
       'Training_score_average_Yes', 'Training_score_average_Yes',
       'No_of_previous_employers_Yes', 'No_of_previous_employers_Yes',
       'Commercial Sales and Marketing_Yes',
       'Commercial Sales and Marketing_Yes',
       'Customer Support and Field Operations_Yes',
       'Customer Support and Field Operations_Yes',
       'Information Technology and Solution Support_Yes',
       'Information Technology and Solution Support_Yes',
       'Information and Strategy_Yes', 'Information and Strategy_Yes',
       'People/HR Management_Yes', 'People/HR Management_Yes',
       'Regulatory and Legal services_Yes',
       'Regulatory and Legal services_Yes', 'Research and Inn

In [0]:
###Hypertune the model intensively with a 5 fold cross validation strategy.
##first grid search for max_depth,min_child_weight then fix those and search for rest in same fashion
### parameter scale_pos_weight is quite important in case of imbalanced dataset

from sklearn.model_selection import GridSearchCV
param_test2b = {
  #'min_child_weight':[5,6],
  #'max_depth': range(3,10,2),
  #'n_estimators':[150,200,300,400],
  #'scale_pos_weight':[1,2,3,4],
  #'colsample_bytree':[0.7,0.8], 
  #'subsample':[0.7,0.8],
  #'gamma':[0,0.2.0.4]
    
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=150, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=3,seed=27), 
 param_grid = param_test2b, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch2b.fit(X_train, y_train)
print(gsearch2b.cv_results_)
print("gsearch2b.best_params_",gsearch2b.best_params_)
print("gsearch2b.best_score_",gsearch2b.best_score_)

{'mean_fit_time': array([90.27193937]), 'std_fit_time': array([31.77764638]), 'mean_score_time': array([0.18961124]), 'std_score_time': array([0.09714284]), 'params': [{}], 'split0_test_score': array([0.5157385]), 'split1_test_score': array([0.50357995]), 'split2_test_score': array([0.53271028]), 'split3_test_score': array([0.48241206]), 'split4_test_score': array([0.51257485]), 'mean_test_score': array([0.50940313]), 'std_test_score': array([0.01646722]), 'rank_test_score': array([1], dtype=int32)}
gsearch2b.best_params_ {}
gsearch2b.best_score_ 0.509403128406278


In [0]:
###Train the model with the best params
modelXg = XGBClassifier(learning_rate=0.1, n_estimators=200, max_depth=4, min_child_weight=7, 
                      gamma=0.4,nthread=4, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',scale_pos_weight=3,seed=29)
modelXg.fit(X_train, y_train)
y_xg = modelXg.predict(X_test)

In [0]:
## Determine whether your model is overfitting or not , with the help of ROC.
print(confusion_matrix(y_test, y_xg))
predictions = [value for value in y_xg]
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
print("Accuracy_score: %.2f%% on test dataset" % (accuracy * 100.0))
print("precision_score: %.2f%% on test dataset" % (precision * 100.0))
print("recall_score: %.2f%% on test dataset" % (recall * 100.0))
print("f1_score: %.2f%% on test dataset" % (f1 * 100.0))
print("roc_auc test set", roc_auc_score(y_test, modelXg.predict_proba(X_test)[:,1]))
print("roc_auc training set", roc_auc_score(y_train, modelXg.predict_proba(X_train)[:,1]))

[[6854  142]
 [ 416  251]]
Accuracy_score: 92.72% on test dataset
precision_score: 63.87% on test dataset
recall_score: 37.63% on test dataset
f1_score: 47.36% on test dataset
roc_auc test set 0.8940886546435187
roc_auc training set 0.9404028780164132


In [0]:
##Once the model is ready, train the model on entire dataset.
modelXg = XGBClassifier(learning_rate=0.1, n_estimators=200, max_depth=4, min_child_weight=7, 
                      gamma=0.4,nthread=4, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',scale_pos_weight=3,seed=29)

modelXg.fit(X, Y)

KeyboardInterrupt: ignored

In [0]:
'''
##find the useful cols
usefulcols = []
for i in sorted( model.booster().get_fscore().items(), key=lambda x: x[1], reverse=True)[:100]:
    usefulcols.append(i[0])
print usefulcols
'''

In [0]:
X_validation = X_validation.drop(duplicate_columns, axis = 1)

In [0]:
X = X_validation.drop(['Promoted_or_Not'],1)
dfvalidation = X_validation.drop(['Promoted_or_Not'],1)
y_valid = modelXg.predict(dfvalidation)
submission = dfvalidation.copy()
submission['Promoted_or_Not'] = y_valid
submission['Promoted_or_Not'].value_counts()

0    15617
1      879
Name: Promoted_or_Not, dtype: int64

In [0]:
submission = submission[['Promoted_or_Not']]
submission.set_index(dfvalidation.index, inplace=True)

from google.colab import files

submission.to_csv('dsn_xgboost_2.csv') 
files.download('dsn_xgboost_2.csv')
